<a href="https://colab.research.google.com/github/Tanuja2725/deeplearning/blob/main/chest_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
# Unmount drive, if mounted:

from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 142  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


import libraries


In [ ]:

%reset -f

import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#  Libraries for building sequential CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input

# Keras has three backend implementations available: the TensorFlow,
#    the Theano, and CNTK backend.
from tensorflow.keras import backend as K

# Save CNN model configuration
from tensorflow.keras.models import model_from_json

# OS related
import os

# For ROC plotting
import matplotlib.pyplot as plt
import numpy as np
# conda install scikit-learn
from sklearn import metrics
import time
# 
from PIL import Image                  # Needed in Windows
import pandas as pd

In [ ]:
# Mount drive at /gdrive:

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# 1.9 Make two folders, one for .tar.gz file
#     and the other for unzipped files
! mkdir /root/chestxray
! mkdir /root/tanuja


mkdir: cannot create directory ‘/root/chestxray’: File exists
mkdir: cannot create directory ‘/root/tanuja’: File exists


In [ ]:
CXRAY=pd.read_csv("/gdrive/MyDrive/colab_data_files/chest_xray/")

IsADirectoryError: ignored

In [ ]:
# 1.10 Copy tar.gz file from Google drive to Virtual machine
! cp /gdrive/MyDrive/colab_data_files/chest_xray/*.*  /root/chestxray

# 1.11 And check
! ls /root/chestxray


cp: cannot stat '/gdrive/MyDrive/colab_data_files/chest_xray/*.*': No such file or directory


In [ ]:
# 1.12 Untar file to folder /root/tanuja/ :

! tar -xvf /root/chestxray/_MACOSX  -C /root/tanuja

tar: /root/chestxray/_MACOSX: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
#%%                            B. Define constants

# 2. Our constants
# 2.1 Dimensions to which our images will be adjusted
img_width, img_height = 250, 250

# 2.2 Data folder containing all training images
train_data_dir = '/root/tanuja/chestxray/train'
test_data_dir =  '/root/tanuja/chestxray/test'

# 2.3 What is the total number of training images
#      that should be generated (not what are available)
nb_train_samples = 2000   # Actual: 1000 + 1000 (more) =    2000

# 2.4 Data folder containing all validation images

validation_data_dir = '/root/tanuja/chestxray/validation'

# 2.5 What is the total no of validation samples that should
#     be generated?
nb_validation_samples = 800   # Actual: 400 + 400 (more) =  800

# Some hyperparameters

# 2.6 Batch size to train at one go:
batch_size = 32             # No of batches = 4000/125 = 32
                            # So per epoch we have 32 batches

# 2.7 How many epochs of training?
epochs = 8                  # For lack of time, let us make it just 5.

# 2.8 No of test samples
test_generator_samples = 300

# 2.9 For test data, what should be batch size
test_batch_size = 16    # This is different from training batch size



In [ ]:
# 3. About keras backend
# 3.1 Can get backend configuration values, as:
K.image_data_format()          # Read .keras conf file to findout
K.backend()

# 3.2 What is our backend and input_shape? Decide data shape as per that.
#     Depth goes last in TensorFlow back-end, first in Theano
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:                                         # So, Tensorflow!
    input_shape = (img_width, img_height, 3)


'channels_last'

'tensorflow'

In [ ]:
model = Sequential()
# 4.1 Add Input layer
model.add(Input(shape = (150,150,3)))
# 4.2 Add Conv2D layer
model.add(Conv2D(
               filters=32,                # For every filter there is set of weights
                                          # For each filter, one bias. So total bias = 32
               kernel_size=(3, 3),        # For each filter there are 3*3=9 kernel_weights
               strides = (1,1),           # So output shape will be 148 X 148 (W-F+1).
                                          # Default strides is 1 only
               #input_shape=input_shape,   # (150,150,3)
               use_bias=True,             # Default value is True
               padding='valid',           # 'valid' => No padding. This is default.
               name="Ist_conv_layer"
               )
         )
 
# 4.4 For each neuron in the convolved network,
#     assign an activation function
#     What is relu? See https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/
 
model.add(Activation('relu'))           # max {0,x}
 
# 4.6 pool_size:  max pooling window size: (2,2)
#     Default stride for pool-layer is same as pool_size
#     Here: 2 across and 2 down ie (2,2)
#     https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
 
model.add(MaxPool2D(pool_size=(2, 2)))
 
# 4.7
model.summary()
# 4.8 Input shape is inferred. Default strides is 1.
#     Note: Activation is specified here only
#     input_shape from top = 74 X 74 X 32
 
model.add(Conv2D(32,
                (3, 3),
                activation = 'relu',
                name = "IInd_con_layer"))
 
# 4.9 So how many parameters now?
#     Total weights = (kernel_weights) * (filters_from_earlier_conv) * (filters)  + ToalNoBias
model.summary()
# model.add(Activation('relu'))
 
# 4.10 Add another pooling layer:
 
model.add(MaxPool2D(pool_size=(2, 2)))
 
# 4.11 Summary?
model.summary()
# 4.12 Add another conv layer but with 64 filters
#      Total weights = (kernel_weights) * (filters_from_earlier_conv) * (filters)  + ToalNoBias
 
model.add(Conv2D(64, (3, 3), name = "IIIrd_conv_layer"))
 
# 4.13
model.summary()
# 4.14
model.add(Activation('relu'))
 
# 4.15
model.add(MaxPool2D(pool_size=(2, 2)))
 
# 4.16 Flattens the input. Does not affect the batch size.
#      It merely flattens the earlier layer without adding any weight
#     See summary() next
#     https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
 
model.add(Flatten(name = "FlattenedLayer"))
 
# 4.17
model.summary()
# 4.18 Dense layer having 64 units
#      dimensionality of the output space.
#      Total weights = hidden_neurons * input_size + bias_foreach_hidden_neurons
#      64 * 18496 + 64
#      Most number of weights come from this layer
#      https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
 
model.add(Dense(64))
 
# 4.19
model.summary()
# 4.20
model.add(Activation('relu'))
 
# Dropout
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
model.add(Dropout(0.7))
 
# 4.21
model.summary()

 
# 4.22 Dense layer having 1 unit
#      dimensionality of the output space.
#      Weights = No of input neurons + bias (64+1)
model.add(Dense(1))
 
# 4.23
model.summary()
 
# 4.24
model.add(Activation('sigmoid'))    # tanh vs sigmoid? See Stackoverflow
 
# 4.25 Compile model
model.compile(
              loss='binary_crossentropy',  # Metrics to be adopted by convergence-routine
              optimizer='rmsprop',         # Strategy for convergence?
              metrics=['accuracy'])        # Metrics, I am interested in


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Ist_conv_layer (Conv2D)     (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Ist_conv_layer (Conv2D)     (None, 148, 148, 32)      896       
                                      

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Ist_conv_layer (Conv2D)     (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 IInd_con_layer (Conv2D)     (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 IIIrd_conv_layer (Conv2D)   (None, 34, 34, 64)        1

In [ ]:
#  Image augmentation
#  Define a preprocessing function
def preprocess(img):
    return img


In [ ]:
# Augmentation configuration for training samples

tr_dtgen = ImageDataGenerator(
                              rescale=1. / 255,      # Normalize colour intensities in 0-1 range
                              shear_range=0.2,       # Shear varies from 0-0.2
                              zoom_range=0.2,
                              horizontal_flip=True,
                              preprocessing_function=preprocess
                              )

In [ ]:
#  Config2: Create iterator from 'train_datagen'


train_generator = tr_dtgen.flow_from_directory(
                                               directory="/root/tanuja/chestxray/train",       
                                               target_size=(224,224),  # Resize images
                                               batch_size=batch_size,  # Return images in batches
                                               class_mode='binary'   # Output labels will be 1D binary labels
                                              )                   

FileNotFoundError: ignored

In [ ]:
id_gen = ImageDataGenerator( featurewise_center=True,
                            rotation_range = 15,
                            #zca_whitening = True,
                            width_shift_range = 0.3,
                            height_shift_range = 0.3,
                            rescale = 1.0/255.0,
                            horizontal_flip = True,
                            vertical_flip = True
                            )


In [ ]:

#  Augmentation configuration we will use
#     for validation. Only rescaling of pixels

val_dtgen = ImageDataGenerator(rescale=1. / 255)

In [ ]:

#  validation data
validation_generator = val_dtgen.flow_from_directory(
                                                     validation_data_dir,
                                                     target_size=(img_width, img_height),   # Resize images
                                                     batch_size=batch_size,    # batch size to augment at a time
                                                     class_mode='binary'  # Return 1D array of class labels
                                                     )

FileNotFoundError: ignored

In [ ]:

history = model.fit_generator(
                              # First argument is always data generator
                              generator = train_generator,          
                              # How many batches per epoch?
                              # Can be any number as generator loops indefinitely 
                              steps_per_epoch=nb_train_samples // batch_size, 
                              # No of epochs                                               
                              epochs=epochs,  
                              # Get validation data from validation generator                      
                              validation_data=validation_generator, 
                              verbose = 1,                          
                              validation_steps=nb_validation_samples // batch_size
                              )



NameError: ignored

In [ ]:
    model= Sequential()
    model.add(Input( shape = (250,250,3))) #Input shape = [width, height, color channels]
        
    # Block One
    model.add(Conv2D(filters=16, kernel_size=3, padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())
    model.add(Dropout(0.2))

    # Block Two
    model.add(Conv2D(filters=32, kernel_size=3, padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())
    model.add(Dropout(0.2))
    
    # Block Three
    model.add(Conv2D(filters=64, kernel_size=3, padding='valid'))
    model.add(Conv2D(filters=64, kernel_size=3, padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())
    model.add(Dropout(0.4))

    # Head
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    #Final Layer (Output)
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer ='adam', metrics='binary_accuracy')

    model.summary()